# Set Up

Pontential Data Sets:
https://www.kaggle.com/datasets/danielgrijalvas/movies
https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows (Only 1000, but good data)
